# Offline-to-Online Training

Our model is trained generatively---the observed data log-likelihood is maximized using the EM algorithm. However, our goal is to deploy the model in a predictive setting. We want to predict the most likely future trajectory given (1) any baseline information and (2) the noisy marker values observed so far. The focus of this notebook is to understand how we can adjust the parameters of a generative trajectory model in order to improve the performance on the trajectory prediction task.

## Related Work

The paper by [Raina and Ng (2003)](http://ai.stanford.edu/~rajatr/papers/nips03-hybrid.pdf) describes a hybrid generative/discriminative model. One of the key ideas in this work is the relative importance of random variables in the generative model when applied in a predictive context (i.e. the generative model is used to derive a conditional probability through Bayes rule). On page 3 there is an interesting point: they show that the decision rule for binary classification of UseNet documents can be formulated as a comparison between the sum of log-likelihood terms. They note that if features are extracted from, say, the message title and the message body there are many more log-likelihood terms for the body than there are for the title. The title, however, may be informative for making the decision. The NBC (or more generally any generatively trained classifier) will treat them all equally, however.

## Experimental Setup

The metric of interest is the mean absolute error aggregated within the usual buckets we've defined---(1,2], (2,4], (4,8], and (8,25]. We'll begin by looking at predictions made after observing one year of data (i.e. we will only train a single online-adapted model). We will compare to two baselines. The first baseline will be the predictions made using the MAP estimate of the subtype under full information (i.e. observing all of the individual's pFVC data) and the second baseline will be the MAP estimate of the subtype under one year of data (i.e. the standard conditional prediction obtained via application of Bayes rule to the generative model).

## Methods

For each individual $i$, let $y_i$ denote the vector of observed measurements, $t_i$ the measurement times, and $x_i$ the vector of covariates used in the population and subpopulation models. Each individual is associated with a subtype, which we denote using $z_i \in \{1, \ldots, K\}$. Let $\Phi_1(t_i)$ denote the population feature matrix, $\Phi_2(t_i)$ denote the subpopulation feature matrix, and $\Phi_3(t_i)$ denote the individual-specific long-term effects feature matrix.

In the generative model, we specify the marginal probability of subtype membership and the conditional probability of observed markers given subtype membership. The marginal probability of subtype membership is modeled using softmax multiclass regression:

$$ p(z_i = k \mid w_{1:K}) \propto \exp \{ x_i^\top w_k \}. $$

The conditional probability of a marker sequence given subtype membership is

$$ p(y_i \mid z_i = k, \beta_{1:K}) = \mathcal{N} ( m_i(k), \Sigma_i ), $$

where

$$ m_i(k) = \Phi_1(t_i) \Lambda x_i + \Phi_2(t_i) \beta_k $$
and
$$ \Sigma_i = \Phi_3(t_i) \Sigma_b \Phi_3^\top(t_i) + K_{\text{OU}}(t_i) + \sigma^2 \mathbf{1}. $$

Given some observed data $y_i$, the posterior over subtype membership $z_i$ is

$$ p(z_i = k \mid y_i) \propto p(z_i = k \mid w_{1:K}) p(y_i \mid z_i = k, \beta_{1:K}). $$

## Code

In [1]:
import numpy as np
import pandas as pd

from imp import reload

In [2]:
import sys
sys.path.append('/Users/pschulam/Git/mypy')

In [3]:
np.set_printoptions(precision=4)

### B-spline Basis

This basis is **hard-coded** to implement the exact basis functions used to fit the model in the R code.

In [4]:
from mypy import bsplines

boundaries   = (-1.0, 23.0)
degree       = 2
num_features = 6
basis = bsplines.universal_basis(boundaries, degree, num_features)

### Kernel Function

In [5]:
from mypy.util import as_row, as_col

def kernel(x1, x2=None, a_const=1.0, a_ou=1.0, l_ou=1.0):
    symmetric = x2 is None
    d = differences(x1, x1) if symmetric else differences(x1, x2)
    K = a_const * np.ones_like(d)
    K += ou_kernel(d, a_ou, l_ou)
    if symmetric:
        K += np.eye(x1.size)
    return K

def ou_kernel(d, a, l):
    return a * np.exp( - np.abs(d) / l )

def differences(x1, x2):
    return as_col(x1) - as_row(x2)

In [6]:
x_test = np.linspace(0, 20, 41)
X_test = basis.eval(x_test)
K_test = kernel(x_test, a_const=16.0, a_ou=36.0, l_ou=2.0)

In [7]:
X_test[:5, :]

array([[ 0.6944,  0.2917,  0.0139,  0.    ,  0.    ,  0.    ],
       [ 0.5625,  0.4062,  0.0312,  0.    ,  0.    ,  0.    ],
       [ 0.4444,  0.5   ,  0.0556,  0.    ,  0.    ,  0.    ],
       [ 0.3403,  0.5729,  0.0868,  0.    ,  0.    ,  0.    ],
       [ 0.25  ,  0.625 ,  0.125 ,  0.    ,  0.    ,  0.    ]])

In [8]:
K_test[:5, :5]

array([[ 53.    ,  44.0368,  37.8351,  33.0052,  29.2437],
       [ 44.0368,  53.    ,  44.0368,  37.8351,  33.0052],
       [ 37.8351,  44.0368,  53.    ,  44.0368,  37.8351],
       [ 33.0052,  37.8351,  44.0368,  53.    ,  44.0368],
       [ 29.2437,  33.0052,  37.8351,  44.0368,  53.    ]])

### Softmax Model

In [9]:
import scipy.optimize as opt

from mypy.models import softmax
reload(softmax)

<module 'mypy.models.softmax' from '/Users/pschulam/Git/mypy/mypy/models/softmax.py'>

### Trajectory Model

In [65]:
from scipy.stats import multivariate_normal

a_const = 16.0
a_ou    = 36.0
l_ou    = 2.0

def phi1(x):
    return np.ones((x.size, 1))

def phi2(x):
    return basis.eval(x)

def gp_posterior(tnew, t, y, kern, **kwargs):
    from numpy import dot
    from scipy.linalg import inv, solve
    
    K11 = kern(tnew, **kwargs)
    K12 = kern(tnew, t, **kwargs)
    K22 = kern(t, **kwargs)
    
    m = dot(K12, solve(K22, y))
    K = K11 - dot(K12, solve(K22, K12.T))
    
    return m, K

def trajectory_means(t, x, b, B):
    from numpy import dot
    
    P1 = phi1(t)
    P2 = phi2(t)
    
    m1 = dot(P1, dot(b, x)).ravel()
    m2 = dot(B, P2.T)
    
    return m1 + m2

def trajectory_logl(t, x, y, z, B, b):
    if t.size < 1:
        return 0.0
    
    m = trajectory_means(t, x, b, B)[z]
    S = kernel(t, a_const=a_const, a_ou=a_ou, l_ou=l_ou)
    
    return multivariate_normal.logpdf(y, m, S)

### Load Parameters

In [20]:
b = np.loadtxt('param/pop.dat')
B = np.loadtxt('param/subpop.dat')
W = np.loadtxt('param/marginal.dat')
W = np.r_[ np.zeros((1, W.shape[1])), W ]

In [53]:
from scipy.misc import logsumexp

def model_posterior(t, x1, x2, y, b, B, W):
    k = B.shape[0]
    lp = softmax.regression_log_proba(x2, W)
    lp += np.array([trajectory_logl(t, x1, y, z, B, b) for z in range(k)])
    ll = logsumexp(lp)
    return np.exp(lp - ll)

def model_loglik(t, x1, x2, y, b, B, W):
    k = B.shape[0]
    lp = softmax.regression_log_proba(x2, W)
    lp += np.array([trajectory_logl(t, x1, y, z, B, b) for z in range(k)])
    ll = logsumexp(lp)
    return ll

### Load Data

In [62]:
from copy import deepcopy

def PatientData(tbl):
    pd = {}
    pd['ptid'] = int(tbl['ptid'].values[0])
    pd['t']    = tbl['years_seen_full'].values.copy()
    pd['y']    = tbl['pfvc'].values.copy()
    pd['x1']   = np.asarray(tbl.loc[:, ['female', 'afram']].drop_duplicates()).ravel()
    pd['x2']   = np.asarray(tbl.loc[:, ['female', 'afram', 'aca', 'scl']].drop_duplicates()).ravel()
    pd['x2']   = np.r_[ 1.0, pd['x2'] ]
    return pd

def truncated_data(pd, censor_time):
    obs = pd['t'] <= censor_time
    pdc = deepcopy(pd)
    pdc['t'] = pd['t'][obs]
    pdc['y'] = pd['y'][obs]
    return pdc, pd['t'][~obs]

def run_inference(pd, b, B, W):
    ll = model_loglik(pd['t'], pd['x1'], pd['x2'], pd['y'], b, B, W)
    posterior = model_posterior(pd['t'], pd['x1'], pd['x2'], pd['y'], b, B, W)
    return ll, posterior

In [50]:
pfvc = pd.read_csv('data/benchmark_pfvc.csv')
data = [PatientData(tbl) for _, tbl in pfvc.groupby('ptid')]

In [57]:
ll, pst = run_inference(data[9], b, B, W)
np.round(pst, 3)

array([ 0.   ,  0.   ,  0.   ,  0.01 ,  0.022,  0.117,  0.816,  0.035])

### Online Tuning Algorithm

In [59]:
full_posteriors = np.array([run_inference(d, b, B, W)[1] for d in data])

In [66]:
part_posteriors = np.array([run_inference(truncated_data(d, 1.0)[0], b, B, W)[1] for d in data])

In [77]:
np.sum(full_posteriors * np.log(full_posteriors))

-430.77831436230167

In [76]:
np.sum(full_posteriors * np.log(part_posteriors))

-790.52486194944299

#### Fitting to the Full Posterior

In [81]:
X = np.array([d['x2'] for d in data])
Y = full_posteriors.copy()
k, p = W.shape

def f(w, k=k, p=p, X=X, Y=Y):
    W = w.reshape((k, p))
    return -sum(softmax.regression_ll(xi, yi, W) for xi, yi in zip(X, Y))

def g(w, k=k, p=p, X=X, Y=Y):
    W = w.reshape((k, p))
    G = sum(softmax.regression_ll_grad(xi, yi, W) for xi, yi in zip(X, Y))
    return -G.ravel()

solution = opt.minimize(f, W.ravel(), jac=g, method='BFGS')
solution.x.reshape((k, p))

array([[  0.    ,   0.    ,   0.    ,   0.    ,   0.    ],
       [  1.5932,  -0.539 ,  12.3583,   0.8797,   1.1137],
       [  1.8355,  -0.2656,  11.9278,  -0.8695,  -0.3211],
       [  1.8279,  -1.    ,  12.8038,  -0.5704,   1.7303],
       [  1.7253,  -0.6034,  13.1681,  -0.925 ,   1.2165],
       [ -0.0353,   0.3386,  13.7896,  -1.5672,   1.7802],
       [  1.9339,  -1.4838,  12.6736,  -0.9307,   1.2987],
       [  0.5535,  -1.1074,  12.6253, -16.2041,   1.1487]])

In [83]:
solution.fun

1215.1193204026597